In [31]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

dist = pd.read_csv('../data/times v4.csv')
le = sklearn.preprocessing.LabelEncoder()
le.fit(dist['Origin_tid'])
dist['from_int'] = le.transform(dist['Origin_tid'])
dist['to_int'] = le.transform(dist['Destination_tid'])

num_vehicles = 20

config = {'cnt_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': {'min': 100, 'persent': 0.01},
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10}

def print_solution(data, manager, routing, solution):
    visited = [0 for i in range(cnt_terminals)]
    paths = []
    for vehicle in range(num_vehicles):
        path = []
        i = routing.Start(vehicle)
        while not routing.IsEnd(i):
            i = solution.Value(routing.NextVar(i))
            if i > 0 and i <= cnt_terminals:
                visited[i - 1] = 1
                path.append(i - 1)
        paths.append(path)
    return visited, paths

inf = int(1e4)
cnt_terminals = config['cnt_terminals']
distance_matrix = np.ones((cnt_terminals + 2, cnt_terminals + 2)) * inf
for i, j, w in zip(dist['from_int'], dist['to_int'], dist['Total_Time']):
    distance_matrix[i + 1, j + 1] = w + config['service_time']
    
for i in range(1, cnt_terminals + 1):
    distance_matrix[i, 0] = inf
    distance_matrix[0, i] = config['service_time']
    distance_matrix[i, i] = 0
    distance_matrix[i, cnt_terminals + 1] = 0
    distance_matrix[cnt_terminals + 1, i] = inf
    
distance_matrix[0, cnt_terminals + 1] = 0
distance_matrix[cnt_terminals + 1, 0] = inf
distance_matrix = distance_matrix.astype(int)

penalties = [1 for i in range(cnt_terminals + 2)]
penalties[0] = 0
penalties[-1] = 0

distance_matrix = (distance_matrix * 100).astype(int)
inf *= 100
config['work_time'] *= 100

In [32]:
vrp_data = {'distance_matrix': distance_matrix,
            'num_vehicles': num_vehicles,
            'num_locations': cnt_terminals + 2,
            'depot': 0}

vrp_data['starts'] = [0] * vrp_data['num_vehicles']
vrp_data['ends'] = [int(cnt_terminals + 1)] * vrp_data['num_vehicles']

In [43]:
manager = pywrapcp.RoutingIndexManager(len(vrp_data['distance_matrix']),
                                   vrp_data['num_vehicles'], vrp_data['starts'], vrp_data['ends'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

# Define cost of each arc.

def distance_callback(from_index, to_index):
    """Returns the manhattan distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return vrp_data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    config['work_time'],  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Allow to drop nodes.
penalty = inf
for node in range(1, len(vrp_data['distance_matrix']) - 1):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty * (100 if node < 20 else 1))

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
search_parameters.time_limit.seconds = 100

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)
    
x = print_solution(vrp_data, manager, routing, solution)
print(x[0])
print(sum(x[0]))

[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 